In [7]:
import numpy as np
import os
import glob
from scipy.spatial.distance import euclidean
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.models import model_from_json
#from keras.optimizers import SGD
from scipy.io import loadmat
from scipy.io import savemat
#from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf

In [8]:
import take_input as inp

In [9]:
X, Y = inp.get_training("/home/aayush/speech/VC_CMU_ARCTIC/Z.mat")

In [19]:
# This will reset the computation graph, if some variables are disallowed, run this once
tf.reset_default_graph()

In [20]:
def generator(z, reuse= None):
    # works like a module | see difference between scope and function
    # "reuse" allows to retain the values of variables inside scope on another call
    with tf.variable_scope("gen", reuse= reuse):
        # Leak factor
        alpha = 0.01
        
        # "units" is the number of nodes in the layer
        hidden1 = tf.layers.dense(inputs = z, units = 512)
        # leaky relu like
        hidden1 = tf.maximum(hidden1, alpha*hidden1)
        
        hidden2 = tf.layers.dense(inputs = hidden1, units = 512)
        hidden2 = tf.maximum(hidden2, alpha*hidden2)
        
        hidden3 = tf.layers.dense(inputs = hidden2, units = 512)
        hidden3 = tf.maximum(hidden3, alpha*hidden3)
        
        # "units" is 784 because (28*28) image. And generator generates an image 
        output = tf.layers.dense(inputs = hidden3, units = 25, activation = tf.nn.tanh)
        
        return output

In [21]:
def discriminator(X, reuse=None):
    #print(real_images.get_shape().as_list())
    with tf.variable_scope("dis", reuse= reuse):
        alpha = 0.01
           
        hidden1 = tf.layers.dense(inputs= X, units= 256)
        hidden1 = tf.maximum(hidden1, hidden1*alpha)
        
        hidden2 = tf.layers.dense(inputs= hidden1, units= 256)
        hidden2 = tf.maximum(hidden2, hidden2*alpha)
        
        hidden3 = tf.layers.dense(inputs= hidden2, units= 256)
        hidden3 = tf.maximum(hidden3, hidden3*alpha)
        
        logits = tf.layers.dense(inputs= hidden3, units= 1)
        output = tf.sigmoid(logits)
        
        return output, logits

In [22]:
target_feat = tf.placeholder(tf.float32, shape= [None, 25])
source_feat = tf.placeholder(tf.float32, shape= [None, 25])

In [23]:
generated_feat = generator(source_feat)

In [24]:
D_output_target, D_logits_target = discriminator(target_feat)

In [25]:
print(D_logits_target.get_shape().as_list())

[None, 1]


In [26]:
D_output_generated, D_logits_generated = discriminator(generated_feat, reuse= True)

In [27]:
def loss_func(logits_in, labels_in):
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits= logits_in, labels= labels_in))
    return cost

In [28]:
D_loss_target = loss_func(D_logits_target, tf.ones_like(D_logits_target))
D_loss_generated = loss_func(D_logits_generated, tf.zeros_like(D_logits_target))

In [29]:
D_loss = D_loss_target + D_loss_generated

In [30]:
G_loss = loss_func(D_logits_generated, tf.ones_like(D_logits_generated))

In [31]:
tvars = tf.trainable_variables()

d_vars = []
g_vars = []

for i in range(len(tvars)):
    if tvars[i].name[:3] == "gen":
        g_vars.append(tvars[i])
    else:
        d_vars.append(tvars[i])

In [32]:
learning_rate = 0.001

In [33]:
D_trainer = tf.train.AdamOptimizer(learning_rate).minimize(D_loss, var_list = d_vars)

In [34]:
G_trainer = tf.train.AdamOptimizer(learning_rate).minimize(G_loss, var_list = g_vars)

In [35]:
epochs = 20

In [36]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(len(X)): 
        for i in range(num_batches):
            batch = mnist.train.next_batch(batch_size)
            batch_images = batch[0].reshape((batch_size, 784))
            batch_images = batch_images*2-1
            
            batch_z = np.random.uniform(-1, 1, size= (batch_size, 100))
            
            _ = sess.run(D_trainer, feed_dict={real_images:batch_images, z: batch_z})
            _ = sess.run(G_trainer, feed_dict={z: batch_z})
            
        print("epoch ", epoch , " completed.")
        sample_z = np.random.uniform(-1, 1, size= (1, 100))
        gen_sample = sess.run(generator(z, reuse=True), feed_dict= {z:sample_z})
        
        samples.append(gen_sample)